In [2]:
from PIL import Image
import numpy as np
import cv2
from collections import Counter


# Open the input image
input_image = Image.open('two.png')

# Get the width and height of the input image
width, height = input_image.size

# Calculate the width of each cropped image
crop_width = width // 8

# Loop through 8 times to crop the image into 8 vertical pieces
for i in range(8):
    # Define the cropping box for the current piece (left, upper, right, lower)
    cropping_box = (i * crop_width, 0, (i+1) * crop_width, height)
    
    # Crop the image using the defined box
    output_image = input_image.crop(cropping_box)
    
    # Save the cropped image
    output_image.save(f'image{i}.png')


image1 = Image.open("image7.png")
image2 = Image.open("image0.png")
image3 = Image.open("image1.png")
image4 = Image.open("image2.png")
image5 = Image.open("image3.png")
image6 = Image.open("image4.png")
image7 = Image.open("image5.png")
image8 = Image.open("image6.png")


width1, height1 = image1.size
width2, height2 = image2.size

width3, height3 = image3.size
width4, height4 = image4.size

width5, height5 = image5.size
width6, height6 = image6.size

width7, height7 = image7.size
width8, height8 = image8.size


total_width1 = width1 + width2
total_width3 = width3 + width4

total_width5 = width5 + width6
total_width7 = width7 + width8



output_image1 = Image.new('RGB', (total_width1, height1))
output_image3 = Image.new('RGB', (total_width3, height3))
output_image5 = Image.new('RGB', (total_width5, height5))
output_image7 = Image.new('RGB', (total_width7, height7))


output_image1.paste(image1, (0, 0))
output_image1.paste(image2, (width1, 0))
output_image1.save("back_image.png")

output_image3.paste(image3, (0, 0))
output_image3.paste(image4, (width3, 0))
output_image3.save("left_image.png")

output_image5.paste(image5, (0, 0))
output_image5.paste(image6, (width5, 0))
output_image5.save("front_image.png")

output_image7.paste(image7, (0, 0))
output_image7.paste(image8, (width7, 0))
output_image7.save("right_image.png")


# Load the pre-trained YOLOv3 model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load the object class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set the minimum probability threshold for object detection
confidence_threshold = 0.5

# Set the non-maximum suppression threshold for overlapping boxes
nms_threshold = 0.4

# Load the four images for object detection
image_files = ["left_image.png", "right_image.png", "front_image.png", "back_image.png"]
sides= ["left side","right side","front side","back side"]
images = []
for image_file in image_files:
    image = cv2.imread(image_file)
    images.append(image)
text=""
# Loop through each image and perform object detection
for i, image in enumerate(images):
    # Prepare the image for object detection
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input for the neural network
    net.setInput(blob)

    # Forward pass through the network
    output_layers = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers)

    # Process the output layer by layer
    boxes = []
    confidences = []
    class_ids = []
    for output in layer_outputs:
        for detection in output:
            # Extract the class scores and box coordinates
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                # Calculate the box coordinates and add to the list
                box = detection[:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
                (center_x, center_y, width, height) = box.astype("int")
                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    list1=[]
    # Print the list of objects detected for the current image
    if len(indices) > 0:
        text +=" on your "+sides[i]+" there are "+ ":\n"
        for j in indices.flatten():
            #print(classes[class_ids[j]], confidences[j])
            list1.append(classes[class_ids[j]])
        
        count_dict=Counter(list1)
        for obj in sorted(count_dict):
            text +=f"{count_dict[obj]}: {obj}\n"
    else:
        text+=" there are No objects on your "+ sides[i]+"\n"
        
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define output text
#text = "Objects detected in image left_image.png : clock 1, person 5. Objects detected in image right_image.png : clock 1, person 6. No objects detected in image front_image.png. Objects detected in image back_image.png : person 6."

# Set properties for the text-to-speech engine
engine.setProperty('rate', 150)
engine.setProperty('volume', 1)

# Speak the output text
engine.say(text)
engine.runAndWait()
print(text)


 on your left side there are :
1: bowl
2: person
1: wine-glass
 on your right side there are :
2: bowl
1: cup
5: person
2: wine-glass
 on your front side there are :
2: cup
2: person
2: wine-glass
 on your back side there are :
3: cup

